# Lógica Computacional 24/25

**Grupo 09**

- João Afonso Almeida Sousa (A102462)
- Rafael Cunha Costa (A102526)


# Inicialização

Para a resolução destes exercícios usamos a biblioteca Python Z3Py que criou uma interface para o Z3.




In [18]:
from z3 import Solver, Int, Or, And, sat, AtLeast

# Problema 1: Horário de uma "StartUp"

1. Pretende-se construir um horário semanal para o plano de reuniões de projeto de uma StartUp de acordo com as seguintes condições:

    a. Cada reunião ocupa uma sala (enumeradas $1...S$) durante um slot (tempo, dia). Assume-se os dias enumerados $1..D$ e, em cada dia, os tempos enumerados $1..T$.

    b. Cada reunião tem associado um projeto (enumerados $1..P$) e um conjunto de participantes. Os diferentes colaboradores são enumerados $1..C$.

    c. Cada projeto tem associado um conjunto de colaboradores, dos quais um  é o líder. Cada projeto realiza um dado número de reuniões semanais. São “inputs” do problema o conjunto de colaboradores de cada projeto, o seu líder e o número de reuniões semanais.

    d. O líder do projeto participa em todas as reuniões do seu projeto; os restantes colaboradores podem ou não participar consoante a sua disponibilidade, num mínimo (“quorum”) de  $50\%$ do total de colaboradores do projeto.  A disponibilidade de cada participante, incluindo o lider,  é um conjunto de “slots” (“inputs” do problema). 

**Análise do problema**

Para criarmos um horário como foi pedido necessitamos de estabelecer certas restrições.


**Condições**

Para a criação do horário, estabelecemos as seguintes condições que verificam a coerência do mesmo:

1. **Validade dos Dias, Tempos e Salas**:
$$
1 \leq \text{dia}_{p,i} \leq D
$$

$$
1 \leq \text{tempo}_{p,i} \leq T
$$

$$
1 \leq \text{sala}_{p,i} \leq S
$$

2. **Disponibilidade do Líder**:
    O líder do projeto \( p \) deve estar disponível no slot (dia, tempo) da reunião \( i
    $$
    \bigvee_{(d,t) \in \text{disponibilidade}_{\text{líder}_p}} (\text{dia}_{p,i} = d \land \text{tempo}_{p,i} = t)
    $$


3. **Quorum de Colaboradores**:
    Pelo menos 50% dos colaboradores do projeto \( p \) devem estar disponíveis no slot (dia, tempo) da reunião \( i \):
    $$
    \text{quorum} = \left\lfloor \frac{\text{número de colaboradores}_p}{2} \right\rfloor
    $$

    $$
    \text{AtLeast}(\text{disponível}_{\text{colab}_1}, \text{disponível}_{\text{colab}_2}, \ldots, \text{disponível}_{\text{colab}_n}, \text{quorum})
    $$

    onde:

    $$
    \text{disponível}_{\text{colab}_j} = \bigvee_{(d,t) \in \text{disponibilidade}_{\text{colab}_j}} (\text{dia}_{p,i} = d \land \text{tempo}_{p,i} = t)
    $$



# Implementação:

**Criação de testes:**

In [20]:
S = 3  #Número de salas
D = 5  #Número de dias
T = 4  #Número de tempos por dia
P = 2  #Número de projetos
C = 6  #Número total de colaboradores

In [21]:
#Projetos existentes
projetos = {
    0: {'lider': 1, 'colaboradores': [1, 2], 'nReunioesPorSemana': 2},
    1: {'lider': 4, 'colaboradores': [4, 5], 'nReunioesPorSemana': 2}
}

In [22]:
#Disponibilidade dos colaboradores
#nDoColaborador: [(dia,tempo)]
disponibilidade = {
    1: [(1, 1), (2, 2), (3,1)],
    2: [(1, 1), (2,2), (3, 1)],
    3: [(1, 1), (3, 2)],
    4: [(2, 2), (4, 3), (3,2)],
    5: [(1, 1), (3, 2), (2,2)],
    6: [(3, 1), (4, 2)]
}

**Implementação do solver e restrições:**

In [19]:
solver = Solver()

In [23]:
#Para cada projeto, criar variaveis para o dia, tempo e sala de cada reunião
reunioes = {}
for p in range(P):
    reunioes[p] = [(Int(f"dia_{p}_{i}"), Int(f"tempo_{p}_{i}"), Int(f"sala_{p}_{i}")) for i in range(projetos[p]['nReunioesPorSemana'])]

In [24]:
#Restrições
for p in range(P):
    for i in range(projetos[p]['nReunioesPorSemana']):
        #Dias, tempos e salas têm de ser válidos
        dia, tempo, sala = reunioes[p][i]
        solver.add(And(dia >= 1, dia <= D))
        solver.add(And(tempo >= 1, tempo <= T))
        solver.add(And(sala >= 1, sala <= S))

        #O líder do projeto tem de estar disponível
        lider = projetos[p]['lider']
        #O slot (dia,tempo) tem de coincidir com pelo menos 1 disponibilidade do líder
        solver.add(Or([And(dia == d, tempo == t) for (d, t) in disponibilidade[lider]]))

        #50% dos colaboradores têm de estar disponíveis
        colaboradores = projetos[p]['colaboradores']
        quorum = len(colaboradores) // 2
        #Filtrar colaboradores com pelo menos 1 disponibilidade no slot escolhido
        disponiveis = [Or([And(dia == d, tempo == t) for (d, t) in disponibilidade[colab]]) for colab in colaboradores]
        solver.add(AtLeast(*disponiveis, quorum))

In [25]:
#Verificar a solução
if solver.check() == sat:
    model = solver.model()
    for p in range(P):
        lider = projetos[p]['lider']
        colaboradores = projetos[p]['colaboradores']
        print(f"Projeto {p} (líder: {lider}, colaboradores: {colaboradores}):")
        
        for i in range(projetos[p]['nReunioesPorSemana']):
            dia = model[reunioes[p][i][0]]
            tempo = model[reunioes[p][i][1]]
            sala = model[reunioes[p][i][2]]
            print(f"  Reunião {i+1}: Dia {dia}, Tempo {tempo}, Sala {sala}")
            
            #Imprimir o número dos participantes da reunião
            participantes = []
            
            #Verificar quem participa e imprimir o número
            for colaborador in colaboradores:
                disponibilidade_colab = disponibilidade[colaborador]
                if any(dia == d and tempo == t for (d, t) in disponibilidade_colab):
                    participantes.append(colaborador)
            
            print(f"    Participantes: {participantes}")
else:
    print("Solução não encontrada!!")

Projeto 0 (líder: 1, colaboradores: [1, 2]):
  Reunião 1: Dia 2, Tempo 2, Sala 1
    Participantes: [1, 2]
  Reunião 2: Dia 3, Tempo 1, Sala 1
    Participantes: [1, 2]
Projeto 1 (líder: 4, colaboradores: [4, 5]):
  Reunião 1: Dia 4, Tempo 3, Sala 1
    Participantes: [4]
  Reunião 2: Dia 3, Tempo 2, Sala 1
    Participantes: [4, 5]
